In [11]:
import cv2
from ultralytics import YOLO
from tracker import Tracker
import torch

torch.cuda.set_device(0)

# Carregar o modelo YOLO
model = YOLO('yolov9c.pt')

# Lista de classes
class_list = [
    'person', 'bicycle', 'car', 'motorcycle', 'bus', 'train', 'truck', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat'
]

# Inicializar o rastreador
tracker = Tracker()
frame_count = 0

# Captura de vídeo
cap = cv2.VideoCapture('avenida.mp4')

# Dicionário e conjunto para rastreamento de carros
cars_down = {}
counter_down = set()

# Loop principal
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Previsão com YOLO
    results = model.predict(frame)[0]
    detections = results.boxes.data.detach().cpu().numpy()
    
    # Filtrar detecções de carros
    cars = [
        [int(row[0]), int(row[1]), int(row[2]), int(row[3]), row[4], int(row[5])] 
        for row in detections if class_list[int(row[5])] == 'car'
    ]

    # Atualizar o rastreador
    tracked_cars = tracker.update([car[:4] for car in cars])

    # Desenhar caixas delimitadoras e IDs
    for (x1, y1, x2, y2, car_id), car in zip(tracked_cars, cars):
        confidence = car[4]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {car_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 255, 255), 2)
        cv2.putText(frame, f'Conf: {confidence:.2f}', (x1, y1 - 30), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 255, 255), 2)

        # Linha vermelha e contagem de carros
        line_y = 308
        offset = 7

        if line_y - offset < cy < line_y + offset:
            cars_down[car_id] = cy
            counter_down.add(car_id)

    # Desenhar linha vermelha e contagem de carros
    cv2.line(frame, (282, 308), (1004, 308), (0, 0, 255), 3)
    cv2.putText(frame, f'going down - {len(counter_down)}', (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

    # Exibir o frame
    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()



0: 640x640 10 cars, 1 bus, 1 truck, 448.8ms
Speed: 6.0ms preprocess, 448.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 10 cars, 1 bus, 1 truck, 449.8ms
Speed: 8.0ms preprocess, 449.8ms inference, 25.9ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 9 cars, 1 bus, 1 truck, 447.8ms
Speed: 8.0ms preprocess, 447.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 8 cars, 1 bus, 1 truck, 450.8ms
Speed: 9.0ms preprocess, 450.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 8 cars, 1 bus, 1 truck, 448.8ms
Speed: 8.0ms preprocess, 448.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 8 cars, 1 bus, 1 truck, 448.8ms
Speed: 7.0ms preprocess, 448.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640 8 cars, 1 bus, 1 truck, 447.8ms
Speed: 7.0ms preprocess, 447.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
0: 640x640

IndexError: list index out of range